In [ ]:
!pip install optuna==3.2.0
!pip install gymnasium==0.28.1
!pip install renderlab
!pip install keras==2.13.1
!pip install tensorflow==2.13.0rc2
!pip install tensorflow-estimator==2.13.0
!pip install tensorflow-macos==2.13.0rc2
!pip install tensorboard==2.13.0


In [ ]:
from typing import Tuple, List, Callable, Union, Optional, Dict
import numpy as np
import os
import pathlib
import imageio
from pathlib import Path
import pandas as pdbb
import gymnasium as gym
import numpy as np
from tqdm import tqdm
from matplotlib import pyplot as plt
from pathlib import Path
from gym.wrappers import RecordVideo
import datetime
import tensorflow as tf
import numpy as np
from tensorflow import keras
from collections import deque
import time
import random
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
import os
import optuna
from optuna.importance import get_param_importances
import uuid
import renderlab as rl
import gc

In [ ]:
def evaluate(
    agent,
    env,
    n_episodes: int,
    observation_space_size: int
) -> Tuple[List, List]:

    # keep track of the reward and steps per episode
    reward_per_episode = []
    steps_per_episode = []

    for i in range(0, n_episodes):

        # reset and initial state from environment needs reshape to np array (tuple instead of array with dimensions 1,4)
        state = np.reshape(env.reset()[0], [1, observation_space_size])

        # initialize the cumulative steps and the rewards for this episode
        rewards = 0
        steps = 0

        # done = environment ended badly / environment ended after 500 episodes
        done = False
        truncated = False

        while not (done or truncated):

            # determine action based on the state
            action = agent.act(state)

            # take the action and observe the reward, new state, etc
            new_state, reward, done, truncated, info = env.step(action)

            # enchange the reward signal
            reward = enhance_reward_signal(new_state, reward, done, truncated, steps)

            # add the data from this step to the episode data
            rewards += reward
            steps += 1

            # reshape the new state
            state = np.reshape(new_state, [1, observation_space_size])

        reward_per_episode.append(rewards)
        steps_per_episode.append(steps)

    return reward_per_episode, steps_per_episode

In [ ]:
# random agent
class RandomAgent:
    def __init__(self, env):
        self.env = env

    def act(self, state: np.array) -> int:
        return self.env.action_space.sample()

In [ ]:
def enhance_reward_signal(next_state, reward, done, truncated, steps):
    if done:
        reward = -100
    elif truncated:
        reward = 100
    elif next_state[0] > 0.2:
        reward = -5
    elif next_state[0] < -0.2:
        reward = -5

    return reward

In [ ]:
def record_random_agent():
    # trying the random agent first
    env = gym.make('CartPole-v1',render_mode = "rgb_array")
    env = rl.RenderFrame(env, "./output")

    observation_space_size = env.observation_space.shape[0]

    # create a random agent
    random_agent = RandomAgent(env)

    # evaluate the random agent
    rewards, steps = evaluate(random_agent, env, 5, observation_space_size)

    env.play()

    # evaluation results
    median_steps = np.median(steps)
    mean_steps = np.mean(steps)
    std_steps = np.std(steps)

    print(f'median steps = {median_steps}')
    print(f'std steps    = {std_steps}')
    print(f'mean steps   = {mean_steps}')

    return rewards, steps

In [ ]:
def record_trained_agent(agent, env):
    env = rl.RenderFrame(env, "./output")

    observation_space_size = env.observation_space.shape[0]

    # evaluate the random agent
    rewards, steps = evaluate(agent, env, 1, observation_space_size)

    env.play()

In [ ]:
# record a random agent
rewards, steps = record_random_agent()

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

fig, ax = plt.subplots(figsize = (10, 4))
ax.set_title("Steps")
pd.Series(steps).plot(kind='hist', bins=100)

plt.show()

In [ ]:
def sample_hyper_parameters(trial: optuna.trial.Trial) -> Dict:

    batch_size = trial.suggest_categorical("batch_size", [16, 32, 64, 128])
    memory_size = trial.suggest_int("memory_size", 500, 10000)
    gamma = trial.suggest_float('gamma', 0.9, 0.99)
    exploration_decay = trial.suggest_categorical('exploration_decay', [0.9, 0.95, 0.98, 0.99])
    layer_size = trial.suggest_categorical('layer_size',[16,32,64,128,256, 512])
    exploration_min = trial.suggest_float('exploration_min',0.001, 0.2)
    episodes = trial.suggest_int("episodes", 100, 200)
    learning_rate = trial.suggest_categorical('learning_rate',[0.001,0.0001])
    extra_intermediate_layers = trial.suggest_categorical('extra_layers',[0,1])
    update_model_steps = trial.suggest_categorical('update_model_steps',[10, 20, 30])

    return {
        'batch_size': batch_size,
        'memory_size': memory_size,
        'gamma': gamma,
        'exploration_decay' : exploration_decay,
        'layer_size' : layer_size,
        'exploration_min' : exploration_min,
        'episodes' : episodes,
        'learning_rate' : learning_rate,
        'extra_intermediate_layers' : extra_intermediate_layers,
         'update_model_steps' : update_model_steps
    }

In [ ]:
import gc
from keras.callbacks import Callback

class MemoryClear(Callback):
    def on_epoch_end(self, epoch, logs=None):
        gc.collect()

In [ ]:
memory_clear_callback = MemoryClear()

class DeepQAgent:

    def __init__(self, env, memory_size, gamma, exploration_decay, layer_size, batch_size, exploration_min, learning_rate, update_model_steps = 20, extra_intermediate_layers = 0, exploration_rate = 1.0):
        self.__env = env
        self.__state_size = 4                                           # we have an 4 numbers that represent our size and this will be the input for our neural net
        self.__action_size = 2                                          # we have 2 possible actions (push the card to the left or to the right)
        self.__memory_size = memory_size
        self.__memory = deque(maxlen=memory_size)                       # memory for storing our experiences
        self.__layer_size_1 = layer_size                                # nn layer 1 width
        self.__layer_size_2 = layer_size                                # nn layer 2 width
        self.__extra_intermediate_layers = extra_intermediate_layers    # extra intermediate layers?
        self.__learning_rate = learning_rate                            # learning rate
        self.__model = self._build_model()                              # our model
        self.__target_model = self._build_model()                       # our target model
        self.__gamma = gamma                                            # discount rate
        self.__exploration_rate = exploration_rate                      # exploration rate
        self.__exploration_min = exploration_min                        # min value for exploration
        self.__exploration_decay = exploration_decay                    # decay in the exploration rate (moving from exploration to exploitation)
        self.__sample_batch_size = batch_size                           # how much samples to we take for the learning step
        self.__uuid = str(uuid.uuid4())                                 # unique uid for the model
        self.__update_model_steps = update_model_steps                  # interval for synchronisation models
        self.__internal_learn_counter = 0                               # learn counter that we use for synchronisation between model and target model

    def add_to_memory(self, state, action, reward, next_state, done):
        # add data to the memory
        self.__memory.append((state, action, reward, next_state, done))

    def disable_exploration(self):
        # disable exploration and go for exploitation
        self.__exploration_rate = 0
        self.__exploration_min = 0

    def reset_episode(self):
        self.__internal_step_counter = 0

    def get_agent_uuid(self):
        # return unique uuid for each created agent
        return self.__uuid

    def _synch_models(self):
        # update the target model with the weights from our model (which is updated every step)
        self.__target_model.set_weights(self.__model.get_weights())

    def save_model(self):
        # save the model in case you want to load it later
        self.__model.save(f'model-{self.__uuid}.keras')

    def replay(self):
        if len(self.__memory) <= self.__sample_batch_size:
            # we wait until there is enough data in the memory before we start the learning proces
            return
        else:
            # sample a random mini batch from the memory
            mini_batch = random.sample(self.__memory, self.__sample_batch_size)

            # build data structures for storing the data from the batch size
            current_state = np.zeros((self.__sample_batch_size, self.__state_size))
            next_state = np.zeros((self.__sample_batch_size, self.__state_size))
            action = np.zeros(self.__sample_batch_size, dtype=int)
            reward = np.zeros(self.__sample_batch_size)
            done = np.zeros(self.__sample_batch_size,dtype=bool)

            # fill the data structures with the data from our memory (tuples)
            for i in range(self.__sample_batch_size):
                current_state[i] = mini_batch[i][0]   # state
                action[i] = mini_batch[i][1]          # action
                reward[i] = mini_batch[i][2]          # reward
                next_state[i] = mini_batch[i][3]      # next_state
                done[i] = mini_batch[i][4]            # done

            # we use double qlearning to stability reasons
            # https://ojs.aaai.org/index.php/AAAI/article/view/10295

            # use the first state to predict both q values
            main_qvalues_state = self.__model.predict(current_state,verbose=0)

            # use the next state to predict both q values
            main_qvalues_new_state = self.__model.predict(next_state,verbose=0)

            # use the next state to predict both q value, but this time we use the target model
            target_qvalues_new_state = self.__target_model.predict(next_state,verbose=0)

            # for each sample in the mini batch
            for i in range(self.__sample_batch_size):
                if done[i]:
                    # no need to look further , there is no next step
                    main_qvalues_state[i][action[i]] = reward[i]
                else:
                     # return the index of the best action (highest q-value for our new state)
                    max_action_from_main_model = np.argmax(main_qvalues_new_state[i])

                    main_qvalues_state[i][action[i]] = reward[i] + self.__gamma * target_qvalues_new_state[i][max_action_from_main_model]

            # we ask the model to optimize itself using the first state and information that we gained through experience
            self.__model.fit(current_state, main_qvalues_state, batch_size=self.__sample_batch_size, epochs=1, verbose=0,callbacks=[memory_clear_callback])

            # increment the step counter and model synchronization logic
            if self.__internal_step_counter % self.__update_model_steps == 0:
                self._synch_models()

            # update the counter
            self.__internal_learn_counter +=1

            # we use the decay factor to reduce our chance of exploration if we are not on the lower limit
            if self.__exploration_rate > self.__exploration_min:
                self.__exploration_rate *= self.__exploration_decay


    def act(self, state: np.array) -> int:
        if np.random.rand() < self.__exploration_rate:
            # take a random action
            random_action = self.__env.action_space.sample()
            return random_action
        else:
            # use the model to make a prediction
            actions = self.__model.predict(state, verbose=0)             # get a model prediction
            best_action = np.argmax(actions[0])                          # get the output index with the highest value
            return best_action

    def _build_model(self):
        model = Sequential()
        model.add(Dense(self.__layer_size_1, input_dim=self.__state_size, activation='relu'))
        model.add(Dense(self.__layer_size_2, activation='relu'))
        if (self.__extra_intermediate_layers > 0):
            for i in range(self.__extra_intermediate_layers + 1):
                model.add(Dense(self.__layer_size_2, activation='relu'))
        model.add(Dense(self.__action_size, activation='linear'))
        model.compile(loss='mse', optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=self.__learning_rate))
        return model

In [ ]:
# train loop for the deep q agent
def optimize_train(agent, env, observation_space_size, n_episodes):
    # print('starting traing of agent ...')

    for i in range(0, n_episodes):
        # reset and reshape the state
        state = np.reshape(env.reset()[0], [1, observation_space_size])

        print(f'new train episode {i} on : {datetime.datetime.now()}')

        # reset the internal state (learning counter)
        agent.reset_episode()

        # initialize done (fail) and truncated (success)
        done = False
        truncated = False

         # execute the garbase collector
        gc.collect()

        # clear all the memory
        keras.backend.clear_session()

        # logging purpose counter
        steps = 0

        while not (done or truncated):

            steps += 1

            # take action based on the state
            action = agent.act(state)

            # observe the signal from the environment after the action
            next_state, reward, done, truncated, info = env.step(action)

            # enhance the signal and merge the truncated and done signals
            reward = enhance_reward_signal(next_state, reward, done, truncated, steps)

            if done or truncated:
                print(f'ended training episode with done {done} - truncated {truncated} - reward {reward} - step count {steps}')

            # reshape the state
            next_state = np.reshape(next_state, [1, observation_space_size])

            # add the observation to the memory
            agent.add_to_memory(state, action, reward, next_state, done)

            # replay observations from the memory and learn if needed
            agent.replay()

            # replace the current state
            state = next_state

In [ ]:
def objective(trial: optuna.trial.Trial) -> float:

    # create an environment
    env = gym.make('CartPole-v1',render_mode = "rgb_array")

    # fetch properties from environment
    observation_space_size = env.observation_space.shape[0]
    action_space_size = env.action_space.n

    print(f'observation space size : {observation_space_size}')
    print(f'action space size      : {action_space_size}')

    # get the hyper parameter config
    args = sample_hyper_parameters(trial)

    print(f'optuna run with params started with params : {trial.params}')

    # create the deep agent
    deep_agent = DeepQAgent(
        env,
        memory_size = args['memory_size'],
        gamma = args['gamma'],
        exploration_decay = args['exploration_decay'],
        layer_size = args['layer_size'],
        batch_size = args['batch_size'],
        exploration_min = args['exploration_min'],
        learning_rate = args['learning_rate'],
        update_model_steps = args['update_model_steps'],
        extra_intermediate_layers = args['extra_intermediate_layers']
    )

    print('train loop started')

    # reset all
    env.reset()

    # train loop
    optimize_train(deep_agent,env, observation_space_size,n_episodes=args['episodes'])

    # disable exploitation and only rely on the model for decisions
    deep_agent.disable_exploration()

    print('evaluation started')
    rewards, steps = evaluate(deep_agent, env, 50, observation_space_size)

    # evaluation results
    median_steps = np.median(steps)
    mean_steps = np.mean(steps)
    std_steps = np.std(steps)

    median_reward = np.median(rewards)
    mean_reward = np.mean(rewards)
    std_reward= np.std(rewards)


    print(f'median reward = {median_reward}')
    print(f'std reward    = {std_reward}')
    print(f'mean reward   = {mean_reward}')

    print(f'median steps = {median_steps}')
    print(f'std steps    = {std_steps}')
    print(f'mean steps   = {mean_steps}')

    # record the agent
    record_trained_agent(deep_agent,env)

    # clear all the memory
    keras.backend.clear_session()

    # execute the garbase collector
    gc.collect()

    return median_reward

In [ ]:
# create an optimized model
study_name = "rl-study"
storage_name = "sqlite:///{}.db".format(study_name)
study = optuna.create_study(study_name=study_name, storage=storage_name,direction='maximize',load_if_exists=True)

study.optimize(objective, n_trials=5, gc_after_trial=True)

In [ ]:
# print param importances
print(get_param_importances(study))

In [ ]:
df = study.trials_dataframe()
df.head(20)

In [ ]:
study.best_params